In [ ]:
%pip install numpy pandas tensorflow tensorflow-hub nltk scikit-learn
%pip install tensorflow-text

ERROR: Could not find a version that satisfies the requirement tensorflow-text (from versions: none)
ERROR: No matching distribution found for tensorflow-text


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text  # Needed for ELMo text processing
import nltk
import re
import string

from tensorflow.keras.layers import Dense, Dropout, Input, Lambda
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords

# Download stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


In [ ]:
# Load datasets
true_df = pd.read_csv("C://Users//isaac//Desktop//FakeRealNews//True.csv")
fake_df = pd.read_csv("C://Users//isaac//Desktop//FakeRealNews//Fake.csv")

# Add a label column: 0 for true news, 1 for fake news
true_df["label"] = 1
fake_df["label"] = 0

# Combine the datasets
df = pd.concat([true_df, fake_df], ignore_index=True)

# Shuffle dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display dataset info
df.info()


In [ ]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'\[.*?\]', '', text)  # Remove brackets
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply text cleaning
df["text"] = (df["title"] + " " + df["text"]).astype(str).apply(clean_text)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42, stratify=df["label"])


In [ ]:
elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False)


In [ ]:
# Define input layer
input_text = Input(shape=(), dtype=tf.string)

# Get ELMo embeddings (reshape input for compatibility)
elmo_embedding = Lambda(lambda x: elmo(tf.reshape(x, [-1]))["default"])(input_text)

# Fully connected layers
dense_layer = Dense(128, activation='relu')(elmo_embedding)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

# Compile Model
elmo_model = Model(inputs=input_text, outputs=output_layer)
elmo_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display model summary
elmo_model.summary()


In [ ]:
history = elmo_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
y_pred = (elmo_model.predict(X_test) > 0.5).astype("int32")

# Print evaluation results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
elmo_model.save("elmo_fake_news_model.h5")
# from tensorflow.keras.models import load_model
# elmo_model = load_model("elmo_fake_news_model.h5", custom_objects={"KerasLayer": hub.KerasLayer})
